In [9]:
!pip install pypdf2
!pip install pdf2image

In [ ]:
from requests import get
import json
import os
from bs4 import BeautifulSoup
from urllib.parse import unquote

url_dir = 'scraping/urls'
urls = {}
host = 'http://lib.buet.ac.bd:8080'

for filename in os.listdir(url_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(url_dir, filename)
        with open(file_path, 'r') as f:
            urls[filename.split('.')[0]] = json.load(f)

def fetch_data(url):
    try:
        response = get(url)
        response.raise_for_status()  # Raise an error for bad responses
        return response.text
    except Exception as e:
        print(f"Error fetching data from {url}: {e}")
        return None
    
def pdf_data(html_string):
    soup = BeautifulSoup(html_string, 'html.parser')
    pdf_link = soup.find(class_='file-link')
    l = pdf_link
    if pdf_link:
        pdf_link = pdf_link.findChild('a')
        pdf_link = pdf_link.get('href')
    else:
        print("PDF link not found.")
        return None
    print(pdf_link)
    file_name = unquote(pdf_link.split('.pdf')[0].split('/')[-1]) + '.pdf'
    pdf_link = host + pdf_link
    try:
        response = get(pdf_link)
        response.raise_for_status()  # Raise an error for bad responses
        return response.content, file_name
    except Exception as e:
        print(f"Error fetching PDF from {pdf_link}: {e}")
        return None


for dept in urls.keys():
    print(dept)
    os.makedirs('scraping/pdf', exist_ok=True)
    for url in urls[dept]:
        pdf,filename = pdf_data(fetch_data(url))
        with open(f'scraping/pdf/{dept},{filename}.pdf', 'wb') as f:
            f.write(pdf)
            print("PDF saved successfully.")
            f.close()


'''pdf = pdf_data(fetch_data(urls['civil'][0]))




with open('scraping/test.pdf', 'wb') as f:
    f.write(pdf)
    print("PDF saved successfully.")
    f.close()
# Example usage'''

In [10]:
import os
from pathlib import Path
import PyPDF2

pdf_directory = Path('pdf')
pdf_files = []

# Check if directory exists
if pdf_directory.exists() and pdf_directory.is_dir():
    # Get all PDF files
    for file in pdf_directory.glob('*.pdf'):
        try:
            pdf_reader = PyPDF2.PdfReader(str(file))
            pdf_files.append({
                'filename': file.name,
                'reader': pdf_reader,
                'pages': len(pdf_reader.pages)
            })
        except Exception as e:
            print(f"Error loading {file.name}: {str(e)}")
else:
    print("Directory 'pdf' not found")

print(f"Loaded {len(pdf_files)} PDF files")

Loaded 98 PDF files


In [11]:
from pdf2image import convert_from_path
import os

# Create output directory if it doesn't exist
images_dir = Path('pdf_images')
images_dir.mkdir(exist_ok=True)

# Add a counter for total pages converted
total_pages = 0

# Convert each PDF file's pages to PNG
for pdf_file in pdf_files:
    try:
        # Get the filename without extension
        base_name = os.path.splitext(pdf_file['filename'])[0]
        
        # Convert PDF to images with better resolution (200 DPI)
        images = convert_from_path(
            str(pdf_directory / pdf_file['filename']),
            dpi=200,
            fmt='png'
        )
        
        # Create subdirectory for each PDF
        pdf_subdir = images_dir 
        
        # Save each page as PNG
        for i, image in enumerate(images):
            image_path = pdf_subdir / f"{base_name},page_{i+1}.png"
            image.save(str(image_path), 'PNG')
            total_pages += 1
            
        print(f"Converted {pdf_file['filename']} - {len(images)} pages")
    except Exception as e:
        print(f"Error converting {pdf_file['filename']}: {str(e)}")

print(f"\nTotal pages converted: {total_pages}")

Converted civil,L-2 T-2, CE, 2021-2022.pdf.pdf - 25 pages
Converted civil,CE-2011-2012 (L-3, T-1) .pdf.pdf - 18 pages
Converted civil,L-1 T-1, CE, 2022-2023.pdf.pdf - 9 pages
Converted civil,L-1 T-1, CE, 2020-2021.pdf.pdf - 17 pages
Converted civil,CE-2014-2015(L-2,T-2).pdf.pdf - 19 pages
Converted civil,2015-2016 (L-2 T-1) -CE.pdf.pdf - 18 pages
Converted civil,L-3,T-1,CE,2018-2019.pdf.pdf - 30 pages
Converted civil,L-4 T-1, CE, 2021-2022.pdf.pdf - 30 pages
Converted civil,L-2,T-1,CE, 2018-2019 1.pdf.pdf - 27 pages
Converted civil,CE-2010-2011 (L-3, T-1) .pdf.pdf - 17 pages
Converted civil,CE-2012-2013 (L-3,T-2).pdf.pdf - 27 pages
Converted civil,L-3T-1, CE, 2020-2021.pdf.pdf - 29 pages
Converted civil,CE L-1, T-2 (2016-2017).pdf.pdf - 23 pages
Converted civil,L-2 T-2, CE, 2019-2020.pdf.pdf - 27 pages
Converted civil,CE-2010-2011 (L-2, T-1) .pdf.pdf - 17 pages
Converted civil,CE-2013-2014 (L-2,T-2).pdf.pdf - 20 pages
Converted civil,CE L-2,T-1 (2016-2017).pdf.pdf - 21 pages
Converted 

KeyboardInterrupt: 